In [ ]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15

import lightgbm as lgb
import xgboost as xgb
import time
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
from sklearn.kernel_ridge import KernelRidge

import tsfresh

import numpy as np
from scipy import signal
from scipy.fftpack import fft, fftshift
import matplotlib.pyplot as plt

In [ ]:
%%time
train = pd.read_csv('input/train_o.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

In [ ]:
train_acoustic_data_small = train['acoustic_data'].values[::50]
train_time_to_failure_small = train['time_to_failure'].values[::50]

fig, ax1 = plt.subplots(figsize=(16, 8))
plt.title("Trends of acoustic_data and time_to_failure. 2% of data (sampled)")
plt.plot(train_acoustic_data_small, color='b')
ax1.set_ylabel('acoustic_data', color='b')
plt.legend(['acoustic_data'])
ax2 = ax1.twinx()
plt.plot(train_time_to_failure_small, color='g')
ax2.set_ylabel('time_to_failure', color='g')
plt.legend(['time_to_failure'], loc=(0.875, 0.9))
plt.grid(False)

del train_acoustic_data_small
del train_time_to_failure_small

In [ ]:
train.shape

In [ ]:
minId_1h = np.array(list(signal.argrelmin(train['time_to_failure'].iloc[:314572740].values)))
minId_2h = np.array(list(signal.argrelmin(train['time_to_failure'].iloc[314572740:].values)))+314572740
minId = np.hstack((minId_1h, minId_2h)).reshape(-1,)

In [ ]:
minId

In [ ]:
%%time
for i in range(len(minId)):
    if i==0:
        train.iloc[:minId[i]+1,:].to_csv('split/train_{:02}.csv'.format(i), index=False)
    else:
        train.iloc[minId[i-1]+1:minId[i]+1,:].to_csv('split/train_{:02}.csv'.format(i), index=False)
i=len(minId)+1
train.iloc[minId[i-1]+1:,:].to_csv('split/train_{:02}.csv'.format(i), index=False)

In [ ]:
np.savetxt('train_split_Id.csv',minId,delimiter=',')